In [14]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import pandas as pd

class VehicleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
        
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')  # 读取图像并转换为RGB格式
        if self.transform:
            image = self.transform(image)  # 如果提供了transform就进行预处理
        label = self.labels[idx]  # 获取对应的标签
        return image, label  # 返回图像张量和标签

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate(model, val_loader, device, verbose=False):
    model.eval()
    all_preds = []
    all_labels = []
    batch_accuracies = []
    batch_f1_scores = []

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            batch_preds = preds.cpu().numpy()
            batch_labels = labels.cpu().numpy()
            
            # 计算每批次的accuracy和macro-F1
            if len(np.unique(batch_labels)) > 1:  # 确保批次中至少有两个不同的类别
                batch_accuracy = accuracy_score(batch_labels, batch_preds)
                batch_f1 = f1_score(batch_labels, batch_preds, average='macro', zero_division=1)
                batch_accuracies.append(batch_accuracy)
                batch_f1_scores.append(batch_f1)
            
            all_preds.extend(batch_preds)
            all_labels.extend(batch_labels)
    
    # 计算整体metrics
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    
    # 计算每批次的方差（如果有足够的批次）
    accuracy_variance = np.var(batch_accuracies) if len(batch_accuracies) > 1 else 0.0
    f1_variance = np.var(batch_f1_scores) if len(batch_f1_scores) > 1 else 0.0
    
    if verbose and len(batch_accuracies) > 0:
        print(f"\nBatch-level statistics:")
        print(f"Number of batches: {len(batch_accuracies)}")
        print(f"Batch Accuracies: {[f'{acc:.4f}' for acc in batch_accuracies]}")
        print(f"Batch Macro-F1 Scores: {[f'{f1:.4f}' for f1 in batch_f1_scores]}")
        print(f"Batch Accuracy mean: {np.mean(batch_accuracies):.4f}, variance: {accuracy_variance:.4f}")
        print(f"Batch Macro-F1 mean: {np.mean(batch_f1_scores):.4f}, variance: {f1_variance:.4f}")

    return accuracy, macro_f1, accuracy_variance, f1_variance

In [15]:
def main():
    # 设置随机种子
    torch.manual_seed(42)
    np.random.seed(42)
    
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")
    
    # 数据预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # 加载数据
    data_dir = "Data/Data"  # 修正数据路径
    image_paths = []
    labels = []
    
    # 遍历文件夹收集图片路径和标签
    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            # 从文件名中提取类别标签（1_xxx.png -> 0, 2_xxx.png -> 1, 3_xxx.png -> 2）
            class_id = int(filename.split('_')[0]) - 1  # 将1,2,3映射到0,1,2
            image_paths.append(os.path.join(data_dir, filename))
            labels.append(class_id)
    
    # 转换为numpy数组
    image_paths = np.array(image_paths)
    labels = np.array(labels)
    
    print(f"总样本数: {len(labels)}")
    print("类别分布:")
    for i in range(3):
        print(f"类别 {i+1}: {sum(labels == i)} 个样本")
    
    # 5折交叉验证
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    accuracies = []
    f1_scores = []
    all_batch_accuracies = []  # 存储所有fold的batch准确率
    all_batch_f1_scores = []   # 存储所有fold的batch F1分数
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(image_paths, labels)):
        print(f"\nFold {fold + 1}")
        
        # 准备数据集
        train_dataset = VehicleDataset(
            [image_paths[i] for i in train_idx],
            [labels[i] for i in train_idx],
            transform=transform
        )
        val_dataset = VehicleDataset(
            [image_paths[i] for i in val_idx],
            [labels[i] for i in val_idx],
            transform=transform
        )
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        # 模型初始化
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 3)  # 3个类别
        model = model.to(device)
        
        # 损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        best_score = 0
        best_epoch = 0
        patience = 3
        no_improve = 0
        
        # 训练循环 (修正缩进)
        for epoch in range(25):
            train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
            accuracy, macro_f1, accuracy_variance, f1_variance = evaluate(model, val_loader, device, verbose=False)
            
            current_score = 0.7 * accuracy + 0.3 * macro_f1
            
            print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Score: {current_score:.4f}")
            
            if current_score > best_score:
                best_score = current_score
                best_epoch = epoch
                no_improve = 0
                torch.save(model.state_dict(), f'best_model_fold{fold}.pth')
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        
        # 重新加载当前fold的最佳模型再评估
        model.load_state_dict(torch.load(f'best_model_fold{fold}.pth'))
        
        # 记录最终结果（使用verbose=True来显示batch级别的结果）
        final_accuracy, final_macro_f1, accuracy_variance, f1_variance = evaluate(model, val_loader, device, verbose=True)
        accuracies.append(final_accuracy)
        f1_scores.append(final_macro_f1)
        
        print(f"\nFold {fold + 1} Final Results:")
        print(f"Accuracy: {final_accuracy:.4f}")
        print(f"Macro-F1: {final_macro_f1:.4f}")
        print(f"Accuracy Variance: {accuracy_variance:.4f}")
        print(f"Macro-F1 Variance: {f1_variance:.4f}")
        fold_score = 0.7 * final_accuracy + 0.3 * final_macro_f1
        print(f"Fold Score: {fold_score:.4f}")

    # 计算5折交叉验证的平均结果
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)  # 使用标准差代替方差
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)  # 使用标准差代替方差
    
    print("\n最终五折交叉验证结果与指标:")
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Macro-F1: {mean_f1:.4f} ± {std_f1:.4f}")
    
    # 计算最终平均得分
    final_score = 0.7 * mean_accuracy + 0.3 * mean_f1
    print(f"\n最终平均得分: {final_score:.4f}")

if __name__ == "__main__":
    main()

使用设备: cuda
总样本数: 1572
类别分布:
类别 1: 720 个样本
类别 2: 608 个样本
类别 3: 244 个样本

Fold 1


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2389, Accuracy: 0.8762, Macro-F1: 0.7868, Score: 0.8494
Epoch 2, Loss: 0.1887, Accuracy: 0.9429, Macro-F1: 0.9413, Score: 0.9424
Epoch 2, Loss: 0.1887, Accuracy: 0.9429, Macro-F1: 0.9413, Score: 0.9424
Epoch 3, Loss: 0.1140, Accuracy: 0.8413, Macro-F1: 0.8427, Score: 0.8417
Epoch 3, Loss: 0.1140, Accuracy: 0.8413, Macro-F1: 0.8427, Score: 0.8417
Epoch 4, Loss: 0.1124, Accuracy: 0.9683, Macro-F1: 0.9658, Score: 0.9675
Epoch 4, Loss: 0.1124, Accuracy: 0.9683, Macro-F1: 0.9658, Score: 0.9675
Epoch 5, Loss: 0.0697, Accuracy: 0.9778, Macro-F1: 0.9735, Score: 0.9765
Epoch 5, Loss: 0.0697, Accuracy: 0.9778, Macro-F1: 0.9735, Score: 0.9765
Epoch 6, Loss: 0.1110, Accuracy: 0.9365, Macro-F1: 0.9385, Score: 0.9371
Epoch 6, Loss: 0.1110, Accuracy: 0.9365, Macro-F1: 0.9385, Score: 0.9371
Epoch 7, Loss: 0.1144, Accuracy: 0.9333, Macro-F1: 0.9273, Score: 0.9315
Epoch 7, Loss: 0.1144, Accuracy: 0.9333, Macro-F1: 0.9273, Score: 0.9315
Epoch 8, Loss: 0.0527, Accuracy: 0.9746, Macro-F1: 

c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2654, Accuracy: 0.7524, Macro-F1: 0.5885, Score: 0.7032
Epoch 2, Loss: 0.1155, Accuracy: 0.9365, Macro-F1: 0.9234, Score: 0.9326
Epoch 2, Loss: 0.1155, Accuracy: 0.9365, Macro-F1: 0.9234, Score: 0.9326
Epoch 3, Loss: 0.0823, Accuracy: 0.7905, Macro-F1: 0.7869, Score: 0.7894
Epoch 3, Loss: 0.0823, Accuracy: 0.7905, Macro-F1: 0.7869, Score: 0.7894
Epoch 4, Loss: 0.0694, Accuracy: 0.9651, Macro-F1: 0.9598, Score: 0.9635
Epoch 4, Loss: 0.0694, Accuracy: 0.9651, Macro-F1: 0.9598, Score: 0.9635
Epoch 5, Loss: 0.1469, Accuracy: 0.8762, Macro-F1: 0.8812, Score: 0.8777
Epoch 5, Loss: 0.1469, Accuracy: 0.8762, Macro-F1: 0.8812, Score: 0.8777
Epoch 6, Loss: 0.1841, Accuracy: 0.8349, Macro-F1: 0.8257, Score: 0.8321
Epoch 6, Loss: 0.1841, Accuracy: 0.8349, Macro-F1: 0.8257, Score: 0.8321
Epoch 7, Loss: 0.1008, Accuracy: 0.9238, Macro-F1: 0.9151, Score: 0.9212
Early stopping at epoch 7
Epoch 7, Loss: 0.1008, Accuracy: 0.9238, Macro-F1: 0.9151, Score: 0.9212
Early stopping at epoch 7

...

Epoch 1, Loss: 0.2699, Accuracy: 0.7389, Macro-F1: 0.7045, Score: 0.7285
Epoch 2, Loss: 0.1518, Accuracy: 0.7930, Macro-F1: 0.8030, Score: 0.7960
Epoch 2, Loss: 0.1518, Accuracy: 0.7930, Macro-F1: 0.8030, Score: 0.7960
Epoch 3, Loss: 0.1142, Accuracy: 0.9236, Macro-F1: 0.8849, Score: 0.9120
Epoch 3, Loss: 0.1142, Accuracy: 0.9236, Macro-F1: 0.8849, Score: 0.9120
Epoch 4, Loss: 0.1166, Accuracy: 0.8344, Macro-F1: 0.7622, Score: 0.8127
Epoch 4, Loss: 0.1166, Accuracy: 0.8344, Macro-F1: 0.7622, Score: 0.8127
Epoch 5, Loss: 0.1093, Accuracy: 0.9745, Macro-F1: 0.9646, Score: 0.9716
Epoch 5, Loss: 0.1093, Accuracy: 0.9745, Macro-F1: 0.9646, Score: 0.9716
Epoch 6, Loss: 0.1425, Accuracy: 0.9459, Macro-F1: 0.9338, Score: 0.9422
Epoch 6, Loss: 0.1425, Accuracy: 0.9459, Macro-F1: 0.9338, Score: 0.9422
Epoch 7, Loss: 0.0859, Accuracy: 0.9554, Macro-F1: 0.9533, Score: 0.9548
Epoch 7, Loss: 0.0859, Accuracy: 0.9554, Macro-F1: 0.9533, Score: 0.9548
Epoch 8, Loss: 0.0608, Accuracy: 0.9713, Macro-F1: 

c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2757, Accuracy: 0.6210, Macro-F1: 0.5723, Score: 0.6064
Epoch 2, Loss: 0.1322, Accuracy: 0.8694, Macro-F1: 0.8391, Score: 0.8603
Epoch 2, Loss: 0.1322, Accuracy: 0.8694, Macro-F1: 0.8391, Score: 0.8603
Epoch 3, Loss: 0.1003, Accuracy: 0.9490, Macro-F1: 0.9463, Score: 0.9482
Epoch 3, Loss: 0.1003, Accuracy: 0.9490, Macro-F1: 0.9463, Score: 0.9482
Epoch 4, Loss: 0.0672, Accuracy: 0.9013, Macro-F1: 0.8975, Score: 0.9002
Epoch 4, Loss: 0.0672, Accuracy: 0.9013, Macro-F1: 0.8975, Score: 0.9002
Epoch 5, Loss: 0.1463, Accuracy: 0.8471, Macro-F1: 0.8634, Score: 0.8520
Epoch 5, Loss: 0.1463, Accuracy: 0.8471, Macro-F1: 0.8634, Score: 0.8520
Epoch 6, Loss: 0.1291, Accuracy: 0.7134, Macro-F1: 0.6748, Score: 0.7018
Early stopping at epoch 6
Epoch 6, Loss: 0.1291, Accuracy: 0.7134, Macro-F1: 0.6748, Score: 0.7018
Early stopping at epoch 6

Batch-level statistics:
Number of batches: 2
Batch Accuracies: ['0.9688', '0.8750']
Batch Macro-F1 Scores: ['0.9687', '0.6349']
Batch Accuracy m

c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2804, Accuracy: 0.8758, Macro-F1: 0.8718, Score: 0.8746
Epoch 2, Loss: 0.1086, Accuracy: 0.9682, Macro-F1: 0.9643, Score: 0.9670
Epoch 3, Loss: 0.0893, Accuracy: 0.9363, Macro-F1: 0.9253, Score: 0.9330
Epoch 4, Loss: 0.0701, Accuracy: 0.9745, Macro-F1: 0.9689, Score: 0.9728
Epoch 5, Loss: 0.0908, Accuracy: 0.9554, Macro-F1: 0.9388, Score: 0.9504
Epoch 6, Loss: 0.0714, Accuracy: 0.9618, Macro-F1: 0.9632, Score: 0.9622
Epoch 7, Loss: 0.0393, Accuracy: 0.9140, Macro-F1: 0.9029, Score: 0.9107
Early stopping at epoch 7

Batch-level statistics:
Number of batches: 2
Batch Accuracies: ['0.9688', '0.9688']
Batch Macro-F1 Scores: ['0.9687', '0.6593']
Batch Accuracy mean: 0.9688, variance: 0.0000
Batch Macro-F1 mean: 0.8140, variance: 0.0239

Fold 5 Final Results:
Accuracy: 0.9777
Macro-F1: 0.9761
Accuracy Variance: 0.0000
Macro-F1 Variance: 0.0239
Fold Score: 0.9772

最终五折交叉验证结果与指标:
Accuracy: 0.9676 ± 0.0123
Macro-F1: 0.9634 ± 0.0118

最终平均得分: 0.9663
